In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

sys.path.append(os.path.join(".."))

from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, normalize
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

from utils import mod_df, drop_features, inverse_mod_X, inverse_mod_y, apply_savgol_filter, apply_median_filter, apply_maximum_filter, apply_is_zero
from my_model import create_model_1, create_model_1_1, create_model_1_2, create_model_1_3, create_model_2, create_model_2_1

In [ ]:
train_df = pd.read_csv(os.path.join('..', 'data', 'unionTrain.csv'))
test_df = pd.read_csv(os.path.join('..', 'data', 'unionTest.csv'))

In [ ]:
X_train, y_train = mod_df(train_df)
X_test, y_test = mod_df(test_df)

In [ ]:
import multiprocessing

def feature_engineering(df):
    result = df.copy()
    
    # drop features
    result = result.drop(['id', 'timestep'], axis=1)
    # result = drop_features(result, [15, 16, 17, 18, 20, 21, 23, 24])
    
    # add features
    with multiprocessing.Pool(processes=8) as pool:
        FEATURE_COLUMNS = result.columns.to_list()
        gb_tmp = pool.map(apply_savgol_filter, result[FEATURE_COLUMNS].to_numpy())
        gb_tmp2 = pool.map(apply_median_filter, result[FEATURE_COLUMNS].to_numpy())
        gb_tmp3 = pool.map(apply_maximum_filter, result[FEATURE_COLUMNS].to_numpy())
        gb_tmp4 = pool.map(apply_is_zero, result[FEATURE_COLUMNS].to_numpy())
        # for col in FEATURE_COLUMNS:
        #     feature = result[col]
        #     feature = feature.to_numpy()
            # result[f'{col}_savgol'] = apply_savgol_filter(feature, window_size=11, polynomial=2)
            # result[f'{col}_median'] = apply_median_filter(feature)
            # result[f'{col}_max'] = apply_maximum_filter(feature)
            # result[f'{col}_sav_med'] = apply_median_filter(apply_savgol_filter(feature, window_size=21), window_size=5)
            # result[f'{col}_is_zero'] = apply_is_zero(feature)
            
    return result, gb_tmp

X_train_1, gb_tmp = feature_engineering(X_train)
# X_test_1 = feature_engineering(X_test)

In [ ]:
len(gb_tmp)

In [ ]:
X_train_1

In [ ]:
'''
med & max: 1m 35s
all: 2m 16s
'''